# TabPFN Evaluation with SHAP Explanations

This notebook demonstrates the usage of the `tabpfn_evaluate` function for evaluating a dataset with TabPFN, outer cross-validation, and SHAP feature importances.

This notebook uses a public dataset to show a minmal working example. Custom datasets with results from GEMSS feature selector should be processed using *tabpfn_evaluate_custom_dataset_results.ipynb*.


In [ ]:
# Imports
from IPython.display import display, Markdown
import os
import json
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import plotly.express as px
from plotly import io as pio
import ast
from sklearn.utils import shuffle

from gemss.postprocessing.tabpfn_evaluation import tabpfn_evaluate

# Example data
from sklearn.datasets import load_iris, fetch_california_housing

pio.renderers.default = "notebook_connected"  # Ensures plotly plots show in notebooks

In [ ]:
# Choose task: 'classification' or 'regression'
task = "regression"

# Load example datasets
if task == "classification":
    data = load_iris()
    X, y = data.data, data.target
    feature_names = data.feature_names
else:
    data = fetch_california_housing()
    X, y = data.data, data.target
    feature_names = data.feature_names

# Shuffle for better cross-validation splits
X, y = shuffle(X, y, random_state=42)

# Subsample heavily for faster demo
n_samples_allowed = 100
if X.shape[0] > n_samples_allowed:
    X = X[:n_samples_allowed]
    y = y[:n_samples_allowed]
n_features_allowed = 5
if X.shape[1] > n_features_allowed:
    X = X[:, :n_features_allowed]
    feature_names = feature_names[:n_features_allowed]

X_df = pd.DataFrame(X, columns=feature_names)
X_df.info()

## Run TabPFN Evaluation with SHAP

- Outer cross-validation
- Feature scaling (standard, minmax or None)
- SHAP explanations (parameter *'explain'*)
- Prints metrics for each fold

> For large X, SHAP explanations take time. For a quick demo, use a small subset or reduce folds.

In [ ]:
results = tabpfn_evaluate(
    X_df,
    y,
    apply_scaling="standard",
    outer_cv_folds=2,
    tabpfn_kwargs=None,
    random_state=42,
    verbose=True,
    explain=True,
)

## CV Results: Average Metrics

In [ ]:
pd.Series(results["average_scores"])

## Feature Importances (SHAP, Mean Per Fold)
Each dictionary below shows mean absolute SHAP values for features in a CV fold.

In [ ]:
for fold, shap_imp in enumerate(results.get("shap_explanations_per_fold", [])):
    display(Markdown(f"Fold {fold+1} SHAP Feature Importances:"))
    display(pd.Series(shap_imp).sort_values(ascending=False))
    print()